# DSA4264 Project

In [1]:
#Import modules
import emoji
import pandas as pd
import numpy as np
import string
from langdetect import detect

In [3]:
#Read in the data (STORE FILES IN SUBFOLDER CALLED DATA)
data2020_df = pd.read_csv('./data/Reddit-Threads_2020-2021.csv',  lineterminator='\n', encoding='utf8')
data2022_df = pd.read_csv('./data/Reddit-Threads_2022-2023.csv', lineterminator='\n', encoding='utf8')

In [15]:
#print(len(data2020_df))
#print(len(data2022_df))

2663782
1840541


In [5]:
#Data cleaning

def clean_data(df):
  pd.options.mode.copy_on_write = True

  #Remove rows with empty review_text
  df = df[df['text'].notnull()]

  #Remove emoji rows
  df['text'] = df['text'].apply(lambda x: emoji.replace_emoji(x,''))

  #Remove punctuation
  df['text'] = df['text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

  #Remove all rows that has non ASCII characters
  df = df[df['text'].apply(lambda x: all(ord(c) < 128 for c in x))]

  #Set all to lower case
  df['text'] = df['text'].str.lower()
  
  #remove deleted rows
  #Remove comments which are [Removed] or [Deleted]
  df = df[~df['text'].str.contains(r'\[removed\]', na=False)]
  df = df[~df['text'].str.contains(r'\[deleted\]', na=False)]


  '''
  #Remove rows that are not in English
  count = 1
  for index, row in df.iterrows():
      try:
        if detect(row['text']) != 'en':
          df.drop(index, inplace=True)
          print(row['text'])
          print(count)
          count += 1
      except:
        df.drop(index, inplace=True)
        print("error for", row['text'])
  '''

  df = df.reset_index(drop=True)

  return df

In [11]:
#cleaned_2020 = clean_data(data2020_df)

In [12]:
#cleaned_2022 = clean_data(data2022_df)

In [6]:
#Get random sample of 500000
length_2020 = len(data2020_df)
length_2022 = len(data2022_df)
size2020 = int(500000 * (length_2020/(length_2020 + length_2022)))
size2022 = 500000 - size2020

sample_2020 = data2020_df.sample(n=size2020, random_state=42)
sample_2022 = data2022_df.sample(n=size2022, random_state=42)

combined_df = pd.concat([sample_2020, sample_2022], axis=0)
combined_df.reset_index(drop=True, inplace=True)

print(len(combined_df)) 

500000


In [7]:
cleaned_df = clean_data(combined_df)

In [16]:
#print(size2020)
#print(size2022)

298642
201358


In [8]:
print(len(cleaned_df))
print(cleaned_df.describe())

439642
           text            timestamp   username  \
count    439642               439642     439642   
unique   382044               438442      38054   
top     deleted  2020-04-16 06:15:34  [deleted]   
freq      33734                    3      53326   

                                                     link    link_id  \
count                                              439642     439642   
unique                                             439642      68190   
top     /r/singapore/comments/tiw1t8/rsingapore_random...  t3_homxdq   
freq                                                    1       1012   

        parent_id       id subreddit_id  \
count      439642   439642       439642   
unique     332533   439642            3   
top     t3_homxdq  i1hghen     t5_2qh8c   
freq          486        1       396643   

                                               moderation  
count                                              439642  
unique                                  

In [9]:
#Save data to csv (Only need to run once)
#cleaned_df.to_csv('Reddit_cleaned_500k.csv', index=False)